In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalAveragePooling2D, Dropout, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
train_path = 'Waste-Net'
valid_path = 'Waste-Net'

In [3]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1)
valid_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet_v2.preprocess_input,
    validation_split=0.1)

In [4]:
class_subset=['cardboard', 'ewaste', 'glass', 'metal','organic', 'paper', 'plastic']
train_batches= train_generator.flow_from_directory(directory=train_path,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=32, 
                                               shuffle=True,
                                               seed=42)
valid_batches= valid_generator.flow_from_directory(directory=valid_path,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=42)

Found 2857 images belonging to 7 classes.
Found 314 images belonging to 7 classes.


In [13]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [14]:
base_model.trainable=False

In [15]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [16]:
model1 = Sequential()
model1.add(base_model)
model1.add(GlobalAveragePooling2D())
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
Total params: 23,564,800
Trainable params: 0
Non-trainable params: 23,564,800
_________________________________________________________________


In [17]:
model1.add(Dense(units=1024, activation='relu'))  
model1.add(BatchNormalization())                
model1.add(Dropout(0.2))                         
model1.add(Dense(units=512, activation='relu'))                
model1.add(Dropout(0.2))                        
model1.add(Dense(units=7, activation='softmax')) 
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)              

In [10]:
model1.add(Dense(units=128, activation='relu'))                
model1.add(Dropout(0.2))                        
model1.add(Dense(units=7, activation='softmax')) 
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               6

In [18]:
model1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_details1 = model1.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 614s 7s/step - loss: 1.2327 - accuracy: 0.5639 - val_loss: 0.7506 - val_accuracy: 0.7389
Epoch 2/10
90/90 [==============================] - 609s 7s/step - loss: 0.5818 - accuracy: 0.8026 - val_loss: 0.6629 - val_accuracy: 0.7739
Epoch 3/10
90/90 [==============================] - 612s 7s/step - loss: 0.4298 - accuracy: 0.8491 - val_loss: 0.6379 - val_accuracy: 0.7866
Epoch 4/10
90/90 [==============================] - 611s 7s/step - loss: 0.2968 - accuracy: 0.8999 - val_loss: 0.6651 - val_accuracy: 0.8217
Epoch 5/10
90/90 [==============================] - 609s 7s/step - loss: 0.2435 - accuracy: 0.9184 - val_loss: 0.6956 - val_accuracy: 0.8312
Epoch 6/10
90/90 [==============================] - 613s 7s/step - loss: 0.1843 - accuracy: 0.9433 - val_loss: 0.6922 - val_accuracy: 0.8376
Epoch 7/10
90/90 [==============================] - 609s 7s/st

Model 1: ResNet V2

In [5]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [6]:
base_model.trainable=False

In [7]:
model1 = Sequential()
model1.add(base_model)
model1.add(AveragePooling2D(pool_size=(7, 7)))
model1.add(Flatten(name="flatten"))
model1.add(Dense(units=256, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(units=7, activation='softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 1, 2048)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 1799      
Total params: 24,091,143
Trainable params: 526,343
Non-trainable params: 23,564,800
______________________________________

In [8]:
model1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model_details = model1.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 842s 9s/step - loss: 0.8236 - accuracy: 0.7053 - val_loss: 0.5663 - val_accuracy: 0.8057
Epoch 2/10
90/90 [==============================] - 809s 9s/step - loss: 0.4067 - accuracy: 0.8540 - val_loss: 0.7085 - val_accuracy: 0.7707
Epoch 3/10
90/90 [==============================] - 804s 9s/step - loss: 0.3004 - accuracy: 0.8943 - val_loss: 0.5903 - val_accuracy: 0.8248
Epoch 4/10
90/90 [==============================] - 807s 9s/step - loss: 0.2270 - accuracy: 0.9202 - val_loss: 0.7551 - val_accuracy: 0.8248
Epoch 5/10
90/90 [==============================] - 806s 9s/step - loss: 0.1899 - accuracy: 0.9324 - val_loss: 0.8772 - val_accuracy: 0.7771
Epoch 6/10
32/90 [=========>....................] - ETA: 8:20 - loss: 0.1225 - accuracy: 0.9587

KeyboardInterrupt: 

Model 2

In [10]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model2 = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [11]:
model2 = Sequential()
base_model2.trainable=False
model2.add(base_model2)

In [12]:
model2.add(GlobalAveragePooling2D()) 
model2.add(Dense(units=1024, activation='relu'))             
model2.add(Dropout(0.2))                       
model2.add(Dense(units=7, activation='softmax')) 
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 7175      
Total params: 25,670,151
Trainable params: 2,105,351
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
model2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_details2 = model2.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 614s 7s/step - loss: 0.8195 - accuracy: 0.7280 - val_loss: 0.5958 - val_accuracy: 0.8344
Epoch 2/10
90/90 [==============================] - 660s 7s/step - loss: 0.3751 - accuracy: 0.8670 - val_loss: 0.7230 - val_accuracy: 0.8185
Epoch 3/10
90/90 [==============================] - 665s 7s/step - loss: 0.2621 - accuracy: 0.9083 - val_loss: 0.8539 - val_accuracy: 0.7994
Epoch 4/10
90/90 [==============================] - 634s 7s/step - loss: 0.1860 - accuracy: 0.9328 - val_loss: 0.9512 - val_accuracy: 0.8025
Epoch 5/10
90/90 [==============================] - 610s 7s/step - loss: 0.1437 - accuracy: 0.9482 - val_loss: 1.3761 - val_accuracy: 0.7643
Epoch 6/10
90/90 [==============================] - 607s 7s/step - loss: 0.1283 - accuracy: 0.9538 - val_loss: 1.0996 - val_accuracy: 0.7898
Epoch 7/10
42/90 [=============>................] - ETA: 5:15 

Model 3

In [14]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model3 = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [8]:
#base_model3.trainable=True

In [9]:
#tuning_layer_name = 'conv5_block1_preact_bn'
#tuning_layer = base_model3.get_layer(tuning_layer_name)
#tuning_index = base_model3.layers.index(tuning_layer)
#for layer in base_model3.layers[:tuning_index]:
#    layer.trainable =  False

In [15]:
base_model3.trainable=False
for layer in base_model3.layers:
    print(layer,layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x00000211808D3388> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211808D3508> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000211808D38C8> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211808D3888> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000211808DAB08> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x00000211808E3908> False
<tensorflow.python.keras.layers.core.Activation object at 0x00000211808DD648> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000021180935D48> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x0000021180940588> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000021180945C48> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211

In [16]:
model3 = Sequential()
model3.add(base_model3)
model3.add(GlobalAveragePooling2D())
model3.add(Dense(units=7, activation='softmax')) 
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 14343     
Total params: 23,579,143
Trainable params: 14,343
Non-trainable params: 23,564,800
_________________________________________________________________


In [18]:
model3.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model_details3 = model3.fit(x=train_batches, validation_data=valid_batches, epochs=6, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/6
90/90 [==============================] - 618s 7s/step - loss: 1.9416 - accuracy: 0.2328 - val_loss: 1.7196 - val_accuracy: 0.3248
Epoch 2/6
90/90 [==============================] - 605s 7s/step - loss: 1.4330 - accuracy: 0.4697 - val_loss: 1.2625 - val_accuracy: 0.5414
Epoch 3/6
90/90 [==============================] - 605s 7s/step - loss: 1.1440 - accuracy: 0.6223 - val_loss: 1.0325 - val_accuracy: 0.6210
Epoch 4/6
90/90 [==============================] - 608s 7s/step - loss: 0.9765 - accuracy: 0.6829 - val_loss: 0.9188 - val_accuracy: 0.6656
Epoch 5/6
90/90 [==============================] - 605s 7s/step - loss: 0.8618 - accuracy: 0.7270 - val_loss: 0.8489 - val_accuracy: 0.6752
Epoch 6/6
90/90 [==============================] - 609s 7s/step - loss: 0.7801 - accuracy: 0.7543 - val_loss: 0.8159 - val_accuracy: 0.6975


In [20]:
model_details3 = model3.fit(x=train_batches, validation_data=valid_batches, epochs=4, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/4
90/90 [==============================] - 606s 7s/step - loss: 0.7080 - accuracy: 0.7798 - val_loss: 0.7778 - val_accuracy: 0.7229
Epoch 2/4
90/90 [==============================] - 606s 7s/step - loss: 0.6708 - accuracy: 0.7900 - val_loss: 0.7584 - val_accuracy: 0.7166
Epoch 3/4
90/90 [==============================] - 606s 7s/step - loss: 0.6367 - accuracy: 0.7928 - val_loss: 0.7392 - val_accuracy: 0.7325
Epoch 4/4
90/90 [==============================] - 609s 7s/step - loss: 0.6011 - accuracy: 0.8085 - val_loss: 0.7174 - val_accuracy: 0.7389


In [24]:
img1 = load_img('test4.jpg', target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img1)
img_array2=tf.keras.applications.resnet_v2.preprocess_input(img_array)
img_array2 = tf.expand_dims(img_array2, 0)
predictions = model3.predict(img_array2)
predicted_class = class_subset[np.argmax(predictions[0])]
confidence = round(100 * (np.max(predictions[0])), 2)
print(confidence, predicted_class)

62.26 plastic


In [26]:
model3.save('resnet_1_f.h5')

In [27]:
model3.save("resnet_f_1")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: resnet_f_1\assets


In [28]:
base_model3.trainable = True
for layer in base_model3.layers:
    print(layer,layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x00000211808D3388> True
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211808D3508> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000211808D38C8> True
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211808D3888> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000211808DAB08> True
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x00000211808E3908> True
<tensorflow.python.keras.layers.core.Activation object at 0x00000211808DD648> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000021180935D48> True
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x0000021180940588> True
<tensorflow.python.keras.layers.core.Activation object at 0x0000021180945C48> True
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x0000021180949B08> 

In [29]:
tuning_layer_name = 'conv5_block1_preact_bn'
tuning_layer = base_model3.get_layer(tuning_layer_name)
tuning_index = base_model3.layers.index(tuning_layer)
for layer in base_model3.layers[:tuning_index]:
    layer.trainable =  False

In [30]:
for layer in base_model3.layers:
    print(layer,layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x00000211808D3388> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211808D3508> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000211808D38C8> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211808D3888> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000211808DAB08> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x00000211808E3908> False
<tensorflow.python.keras.layers.core.Activation object at 0x00000211808DD648> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000021180935D48> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x0000021180940588> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000021180945C48> False
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x00000211

In [32]:
model3.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model_details3 = model3.fit(x=train_batches, validation_data=valid_batches, epochs=4, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/4
90/90 [==============================] - 763s 8s/step - loss: 0.5407 - accuracy: 0.8253 - val_loss: 0.5595 - val_accuracy: 0.7930
Epoch 2/4
90/90 [==============================] - 762s 8s/step - loss: 0.3832 - accuracy: 0.8880 - val_loss: 0.5142 - val_accuracy: 0.8025
Epoch 3/4
90/90 [==============================] - 760s 8s/step - loss: 0.2830 - accuracy: 0.9300 - val_loss: 0.4806 - val_accuracy: 0.8185
Epoch 4/4
90/90 [==============================] - 761s 8s/step - loss: 0.2132 - accuracy: 0.9541 - val_loss: 0.4685 - val_accuracy: 0.8344


In [34]:
model3.save('resnet_1_t.h5')

In [35]:
model3.save("resnet_t_1")

INFO:tensorflow:Assets written to: resnet_t_1\assets


In [39]:
img1 = load_img('test4.jpg', target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img1)
img_array2=tf.keras.applications.resnet_v2.preprocess_input(img_array)
img_array2 = tf.expand_dims(img_array2, 0)
predictions = model3.predict(img_array2)
predicted_class = class_subset[np.argmax(predictions[0])]
confidence = round(100 * (np.max(predictions[0])), 2)
print(confidence, predicted_class)

52.11 paper


In [40]:
model_details3 = model3.fit(x=train_batches, validation_data=valid_batches, epochs=2, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/2
90/90 [==============================] - 765s 8s/step - loss: 0.1611 - accuracy: 0.9737 - val_loss: 0.4595 - val_accuracy: 0.8408
Epoch 2/2
90/90 [==============================] - 765s 9s/step - loss: 0.1276 - accuracy: 0.9828 - val_loss: 0.4446 - val_accuracy: 0.8567


In [47]:
img1 = load_img('test1.jpg', target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img1)
img_array2=tf.keras.applications.resnet_v2.preprocess_input(img_array)
img_array2 = tf.expand_dims(img_array2, 0)
predictions = model3.predict(img_array2)
predicted_class = class_subset[np.argmax(predictions[0])]
confidence = round(100 * (np.max(predictions[0])), 2)
print(confidence, predicted_class)

92.24 ewaste


Model 4

In [48]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model4 = tf.keras.applications.ResNet50V2(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [49]:
base_model4.trainable=False

In [50]:
model4=Sequential()
model4.add(base_model4)
model4.add(GlobalAveragePooling2D())
model4.add(Dense(units=2048, activation='relu')) 
model4.add(BatchNormalization())
model4.add(Dense(units=512, activation='relu'))
model4.add(Dropout(0.2))                       
model4.add(Dense(units=7, activation='softmax')) 
model4.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                

In [51]:
model4.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
model_details4 = model4.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/5
90/90 [==============================] - 827s 9s/step - loss: 1.1640 - accuracy: 0.7189 - val_loss: 2.1596 - val_accuracy: 0.5955
Epoch 2/5
90/90 [==============================] - 829s 9s/step - loss: 0.4571 - accuracy: 0.8617 - val_loss: 1.4919 - val_accuracy: 0.7325
Epoch 3/5
90/90 [==============================] - 849s 9s/step - loss: 0.3142 - accuracy: 0.9083 - val_loss: 1.7057 - val_accuracy: 0.7484
Epoch 4/5
90/90 [==============================] - 827s 9s/step - loss: 0.1768 - accuracy: 0.9394 - val_loss: 1.9979 - val_accuracy: 0.7389
Epoch 5/5
25/90 [=======>......................] - ETA: 9:45 - loss: 0.1510 - accuracy: 0.9466

KeyboardInterrupt: 